# BombCell with UnitMatch Parameters

This notebook demonstrates how to run BombCell with parameters optimized for UnitMatch.

**Key differences from standard BombCell:**
- Extracts 1000 raw spikes per unit (instead of 100)
- Saves waveforms in UnitMatch format for cross-validation
- Disables waveform detrending
- Ensures data decompression for raw access

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from pathlib import Path
import warnings
warnings.filterwarnings('ignore')

# Import BombCell
import bombcell as bc

print("✅ BombCell imported successfully")

✅ ipywidgets available - interactive GUI ready
✅ BombCell imported successfully


## Set Your Data Paths

Update these paths to point to your actual data:

In [ ]:
# Set your data paths here
ks_dir = "/path/to/your/kilosort/output"  # Path to Kilosort output
raw_file = "/path/to/your/raw/data.bin"   # Path to raw .bin file (optional)
meta_file = "/path/to/your/meta/file.meta" # Path to .meta file (optional)
kilosort_version = 4  # Adjust based on your Kilosort version (2, 3, or 4)

# Output directory for results
save_path = Path(ks_dir) / "qMetrics"

print(f"📁 Kilosort directory: {ks_dir}")
print(f"📁 Results will be saved to: {save_path}")

## Run BombCell with UnitMatch Parameters

In [ ]:
# Run BombCell with UnitMatch parameters
print("🚀 Running BombCell with UnitMatch parameters...")
print("   This will extract 1000 raw spikes per unit and save in UnitMatch format")

quality_metrics, param, unit_type, unit_type_string = bc.run_bombcell_unit_match(
    ks_dir=ks_dir,
    save_path=save_path,
    raw_file=raw_file,
    meta_file=meta_file,
    kilosort_version=kilosort_version
)

print("✅ BombCell analysis complete!")

## View Results

In [ ]:
# Display results summary
print(f"📊 BombCell Results:")
print(f"   Total units analyzed: {len(unit_type)}")
print(f"   Good units: {np.sum(unit_type_string == 'GOOD')}")
print(f"   MUA units: {np.sum(unit_type_string == 'MUA')}")
print(f"   Noise units: {np.sum(unit_type_string == 'NOISE')}")
print(f"   Non-somatic units: {np.sum(unit_type_string == 'NON-SOMA')}")

# Check that UnitMatch waveforms were saved
raw_waveforms_dir = save_path / "RawWaveforms"
if raw_waveforms_dir.exists():
    waveform_files = list(raw_waveforms_dir.glob("Unit*_RawSpikes.npy"))
    print(f"   ✅ Saved {len(waveform_files)} UnitMatch waveform files")
    print(f"   📁 UnitMatch waveforms directory: {raw_waveforms_dir}")
else:
    print(f"   ⚠️  No UnitMatch waveforms found. Check that raw data file is accessible.")

In [ ]:
# Convert results to DataFrame for easier viewing
results_df = pd.DataFrame(quality_metrics)
results_df['unit_type'] = unit_type_string
results_df['cluster_id'] = results_df['phy_clusterID'].astype(int)

# Display first few rows
print("📋 Quality Metrics Summary:")
display(results_df.head())

# Show unit type distribution
print("\n📊 Unit Type Distribution:")
print(results_df['unit_type'].value_counts())

## Key UnitMatch Parameters Used

The following parameters were automatically set for UnitMatch compatibility:

In [ ]:
print("🔧 UnitMatch Parameters Used:")
print(f"   detrendWaveform: {param['detrendWaveform']} (disabled for UnitMatch)")
print(f"   nRawSpikesToExtract: {param['nRawSpikesToExtract']} (more spikes for better matching)")
print(f"   saveMultipleRaw: {param['saveMultipleRaw']} (saves cross-validation waveforms)")
print(f"   decompress_data: {param['decompress_data']} (ensures raw data access)")

## Files Generated

This analysis creates the following files that are compatible with UnitMatch:

1. **Standard BombCell outputs:**
   - Quality metrics CSV and parquet files
   - Unit classification results
   - Summary plots

2. **UnitMatch-specific outputs:**
   - `RawWaveforms/Unit{ID}_RawSpikes.npy`: Cross-validation waveforms for each unit
   - Raw waveform data with proper formatting for UnitMatch analysis

